In [ ]:
# This is the third step of the tweets preprocessing procedure

In [1]:
import pandas as pd

# print non truncated column info in pandas dataframe
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', 500)

## - Create features based on the lexicons

In [2]:
# read the preprocessed tweets
df_tweets = pd.read_pickle('results/tweets_step_1_v1.pkl')

In [4]:
df_tweets['text_id'] = df_tweets.groupby(['text']).ngroup()

In [5]:
# drop duplicates
df_tweets2 = df_tweets.drop_duplicates(subset=['text'], keep='first')
#del df_tweets

## - GrAFS lexicon

<li>Sum of subjectivity, positivity, negativity from GrAFS lexicon - 3 features</li>
<li>Sum of fear, disgust, …happiness from GrAFS lexicon. – 6 features</li>

In [8]:
# read the preprocessed GrAFS lexicon
df_GrAFS = pd.read_pickle('/home/infilab/SentimentAnalysis/lexicons/fixed_lexicons/df_GrAFS.pkl')

In [9]:
df_GrAFS.head()

,keyword,subj,positive,negative,anger,disgust,fear,happy,sad,surprise,keyword_CAP,keyword2,keyword_STEM
0,εκατοχρονίτικούς,0.375,0.0,0.75,0.1250,0.3125,0.000,0.0,0.0,0.3125,ΕΚΑΤΟΧΡΟΝΙΤΙΚΟΥΣ,εκατοχρονιτικους,ΕΚΑΤΟΧΡΟΝΙΤ
1,επιδρομέων,0.250,0.0,0.50,0.1875,0.0625,0.250,0.0,0.0,0.2500,ΕΠΙΔΡΟΜΕΩΝ,επιδρομεων,ΕΠΙΔΡΟΜ
2,άσεμνου,0.500,0.0,0.75,0.1875,0.3750,0.000,0.0,0.0,0.0000,ΑΣΕΜΝΟΥ,ασεμνου,ΑΣΕΜΝ
3,πυρπολούσε,0.375,0.0,0.50,0.2500,0.1250,0.125,0.0,0.0,0.2500,ΠΥΡΠΟΛΟΥΣΕ,πυρπολουσε,ΠΥΡΠΟΛ
4,τζόβενο,0.625,0.0,0.75,0.1875,0.3125,0.000,0.0,0.0,0.2500,ΤΖΟΒΕΝΟ,τζοβενο,ΤΖΟΒΕΝ


In [10]:
%%capture
# count how many unique words of the tweet are matched with the lexicon
df_tweets2['N_matches'] = df_tweets2['Tokens'].apply(lambda x: 
                                len(df_GrAFS[df_GrAFS.keyword_CAP.isin(x)]))

# find the unique words of each tweet that match with the lexicon
df_tweets2['Match_Terms'] = df_tweets2['Tokens'].apply(lambda x: 
                        df_GrAFS[df_GrAFS.keyword_CAP.isin(x)].keyword_CAP.unique())

In [11]:
# check the number of times each word was matched
matched_words = df_tweets2['Match_Terms'].apply(pd.Series).stack().value_counts()
matched_words[0:5]

ΤΟ     1046
ΤΟΥ    944 
ΤΗΣ    767 
ΤΗΝ    761 
ΜΕ     716 
dtype: int64

In [12]:
print('The number of unique words that were matched with the lexicon is: ', len(matched_words))

The number of unique words that were matched with the lexicon is:  1107


In [13]:
def sum_scores(terms, df, col):
    terms_df = pd.DataFrame({'Terms':terms})
    terms_df = terms_df.merge(df[['keyword_CAP',col]], left_on='Terms', right_on='keyword_CAP', how='left')
    terms_df.dropna(inplace=True)
    #n = len(terms_df)
    sum_score = terms_df[col].sum()
    
    return sum_score

In [14]:
%%capture
# Create 9 columns with the sums of the bellow columns for each tweet
columns_ = ['subj', 'positive', 'negative', 'anger', 'disgust', 'fear',
            'happy', 'sad', 'surprise']

df_GrAFS_unq = df_GrAFS.drop_duplicates(subset=['keyword_CAP'], keep='first')

for column in columns_[0:]:
    col = 'Sum_'+column
    df_tweets2[col] = df_tweets2['Tokens'].apply(lambda x: sum_scores(x, df_GrAFS_unq, column))

In [15]:
df_tweets2.head(2)

,date,hashtags,id,location,mentions,searchParam,text,user_id,user_name,date2,day,Text_only,Text_only2,Text_only3,Tweet_upper,Tokens,Tokens_stem,Bigrams,Bigrams2,N_exclam,N_hash,N_mentions,N_uppers,has_url,N_words,text_id,N_matches,Match_Terms,Sum_subj,Sum_positive,Sum_negative,Sum_anger,Sum_disgust,Sum_fear,Sum_happy,Sum_sad,Sum_surprise
0,2019-09-08 23:39:53,[],1170844235215642624,,[EliasPalai],#ΝΔ,RT @EliasPalai: Πρόθεση Ψήφου Εθνικών Εκλογών\n\n(Και κάτοικοι του Εξωτερικού αφού πλέον στις δημοσκοπήσεις θα πρέπει οι εταιρίες δημοσκοπήσε…,1162444088500772864,Μπιλαρος,2019-09-08,4,RT : Πρόθεση Ψήφου Εθνικών Εκλογών (Και κάτοικοι του Εξωτερικού αφού πλέον στις δημοσκοπήσεις θα πρέπει οι εταιρίες δημοσκοπήσε…,rt : πρόθεση ψήφου εθνικών εκλογών (και κάτοικοι του εξωτερικού αφού πλέον στις δημοσκοπήσεις θα πρέπει οι εταιρίες δημοσκοπήσε…,rt : προθεση ψηφου εθνικων εκλογων (και κατοικοι του εξωτερικου αφου πλεον στις δημοσκοπησεις θα πρεπει οι εταιριες δημοσκοπησε…,RT : ΠΡΟΘΕΣΗ ΨΗΦΟΥ ΕΘΝΙΚΩΝ ΕΚΛΟΓΩΝ (ΚΑΙ ΚΑΤΟΙΚΟΙ ΤΟΥ ΕΞΩΤΕΡΙΚΟΥ ΑΦΟΥ ΠΛΕΟΝ ΣΤΙΣ ΔΗΜΟΣΚΟΠΗΣΕΙΣ ΘΑ ΠΡΕΠΕΙ ΟΙ ΕΤΑΙΡΙΕΣ ΔΗΜΟΣΚΟΠΗΣΕ…,"[RT, ΠΡΟΘΕΣΗ, ΨΗΦΟΥ, ΕΘΝΙΚΩΝ, ΕΚΛΟΓΩΝ, ΚΑΙ, ΚΑΤΟΙΚΟΙ, ΤΟΥ, ΕΞΩΤΕΡΙΚΟΥ, ΑΦΟΥ, ΠΛΕΟΝ, ΣΤΙΣ, ΔΗΜΟΣΚΟΠΗΣΕΙΣ, ΘΑ, ΠΡΕΠΕΙ, ΟΙ, ΕΤΑΙΡΙΕΣ, ΔΗΜΟΣΚΟΠΗΣΕ]","[RT, ΠΡΟΘΕΣ, ΨΗΦ, ΕΘΝ, ΕΚΛΟΓ, ΚΑΙ, ΚΑΤΟΙΚ, ΤΟΥ, ΕΞΩΤΕΡ, ΑΦΟΥ, ΠΛΕΟΝ, ΣΤΙΣ, ΔΗΜΟΣΚΟΠΗΣ, ΘΑ, ΠΡΕΠ, ΟΙ, ΕΤΑΙΡ, ΔΗΜΟΣΚΟΠ]","[(RT, ΠΡΟΘΕΣΗ), (ΠΡΟΘΕΣΗ, ΨΗΦΟΥ), (ΨΗΦΟΥ, ΕΘΝΙΚΩΝ), (ΕΘΝΙΚΩΝ, ΕΚΛΟΓΩΝ), (ΕΚΛΟΓΩΝ, ΚΑΙ), (ΚΑΙ, ΚΑΤΟΙΚΟΙ), (ΚΑΤΟΙΚΟΙ, ΤΟΥ), (ΤΟΥ, ΕΞΩΤΕΡΙΚΟΥ), (ΕΞΩΤΕΡΙΚΟΥ, ΑΦΟΥ), (ΑΦΟΥ, ΠΛΕΟΝ), (ΠΛΕΟΝ, ΣΤΙΣ), (ΣΤΙΣ, ΔΗΜΟΣΚΟΠΗΣΕΙΣ), (ΔΗΜΟΣΚΟΠΗΣΕΙΣ, ΘΑ), (ΘΑ, ΠΡΕΠΕΙ), (ΠΡΕΠΕΙ, ΟΙ), (ΟΙ, ΕΤΑΙΡΙΕΣ), (ΕΤΑΙΡΙΕΣ, ΔΗΜΟΣΚΟΠΗΣΕ)]","[RT ΠΡΟΘΕΣΗ, ΠΡΟΘΕΣΗ ΨΗΦΟΥ, ΨΗΦΟΥ ΕΘΝΙΚΩΝ, ΕΘΝΙΚΩΝ ΕΚΛΟΓΩΝ, ΕΚΛΟΓΩΝ ΚΑΙ, ΚΑΙ ΚΑΤΟΙΚΟΙ, ΚΑΤΟΙΚΟΙ ΤΟΥ, ΤΟΥ ΕΞΩΤΕΡΙΚΟΥ, ΕΞΩΤΕΡΙΚΟΥ ΑΦΟΥ, ΑΦΟΥ ΠΛΕΟΝ, ΠΛΕΟΝ ΣΤΙΣ, ΣΤΙΣ ΔΗΜΟΣΚΟΠΗΣΕΙΣ, ΔΗΜΟΣΚΟΠΗΣΕΙΣ ΘΑ, ΘΑ ΠΡΕΠΕΙ, ΠΡΕΠΕΙ ΟΙ, ΟΙ ΕΤΑΙΡΙΕΣ, ΕΤΑΙΡΙΕΣ ΔΗΜΟΣΚΟΠΗΣΕ]",0,0,1,1,0,18,690,2,[ΤΟΥ],0.125,0.0,0.25,0.0,0.0000,0.0000,0.0000,0.0,0.0
1,2019-09-08 22:46:36,"[ΝΔ, Επιτελικό_κράτος, Μητσοτακη, ΔΕΘ2019, Βελλιδειο]",1170830825153318912,"Athens, Greece",[],#ΝΔ,"Το τερμάτισαν στην ""επικοινωνία"" (η χάρα του PR η #ΝΔ, έχουν και γνωστούς με γραφεία...). Καμιά κατοστάρια μόνο δημοσιογράφους θα προσλάβει το #Επιτελικό_κράτος (κομαντατούρα #Μητσοτακη). Πήγαν για interview; Οποιος χειροκροτήσει πιο δυνατά προσλαμβάνεται;😂 #ΔΕΘ2019 #Βελλιδειο https://t.co/2duOrmL7u5",1142893644082503681,Παναγιώτης Χατζηπλής 🇬🇷,2019-09-08,4,"Το τερμάτισαν στην ""επικοινωνία"" (η χάρα του PR η , έχουν και γνωστούς με γραφεία...). Καμιά κατοστάρια μόνο δημοσιογράφους θα προσλάβει το (κομαντατούρα ). Πήγαν για interview; Οποιος χειροκροτήσει πιο δυνατά προσλαμβάνεται;","το τερμάτισαν στην ""επικοινωνία"" (η χάρα του pr η , έχουν και γνωστούς με γραφεία...). καμιά κατοστάρια μόνο δημοσιογράφους θα προσλάβει το (κομαντατούρα ). πήγαν για interview; οποιος χειροκροτήσει πιο δυνατά προσλαμβάνεται;","το τερματισαν στην ""επικοινωνια"" (η χαρα του pr η , εχουν και γνωστους με γραφεια...). καμια κατοσταρια μονο δημοσιογραφους θα προσλαβει το (κομαντατουρα ). πηγαν για interview; οποιος χειροκροτησει πιο δυνατα προσλαμβανεται;","ΤΟ ΤΕΡΜΑΤΙΣΑΝ ΣΤΗΝ ""ΕΠΙΚΟΙΝΩΝΙΑ"" (Η ΧΑΡΑ ΤΟΥ PR Η , ΕΧΟΥΝ ΚΑΙ ΓΝΩΣΤΟΥΣ ΜΕ ΓΡΑΦΕΙΑ...). ΚΑΜΙΑ ΚΑΤΟΣΤΑΡΙΑ ΜΟΝΟ ΔΗΜΟΣΙΟΓΡΑΦΟΥΣ ΘΑ ΠΡΟΣΛΑΒΕΙ ΤΟ (ΚΟΜΑΝΤΑΤΟΥΡΑ ). ΠΗΓΑΝ ΓΙΑ INTERVIEW; ΟΠΟΙΟΣ ΧΕΙΡΟΚΡΟΤΗΣΕΙ ΠΙΟ ΔΥΝΑΤΑ ΠΡΟΣΛΑΜΒΑΝΕΤΑΙ;","[ΤΟ, ΤΕΡΜΑΤΙΣΑΝ, ΣΤΗΝ, ΕΠΙΚΟΙΝΩΝΙΑ, Η, ΧΑΡΑ, ΤΟΥ, PR, Η, ΕΧΟΥΝ, ΚΑΙ, ΓΝΩΣΤΟΥΣ, ΜΕ, ΓΡΑΦΕΙΑ, ΚΑΜΙΑ, ΚΑΤΟΣΤΑΡΙΑ, ΜΟΝΟ, ΔΗΜΟΣΙΟΓΡΑΦΟΥΣ, ΘΑ, ΠΡΟΣΛΑΒΕΙ, ΤΟ, ΚΟΜΑΝΤΑΤΟΥΡΑ, ΠΗΓΑΝ, ΓΙΑ, INTERVIEW, ΟΠΟΙΟΣ, ΧΕΙΡΟΚΡΟΤΗΣΕΙ, ΠΙΟ, ΔΥΝΑΤΑ, ΠΡΟΣΛΑΜΒΑΝΕΤΑΙ]","[ΤΟ, ΤΕΡΜΑΤΙΣ, ΣΤΗΝ, ΕΠΙΚΟΙΝΩΝ, Η, ΧΑΡ, ΤΟΥ, PR, Η, ΕΧ, ΚΑΙ, ΓΝΩΣΤ, ΜΕ, ΓΡΑΦΕΙ, ΚΑΜ, ΚΑΤΟΣΤΑΡ, ΜΟΝ, ΔΗΜΟΣΙΟΓΡΑΦ, ΘΑ, ΠΡΟΣΛΑΒ, ΤΟ, ΚΟΜΑΝΤΑΤΟΥΡ, ΠΗΓ, ΓΙΑ, INTERVIEW, ΟΠΟΙ, ΧΕΙΡΟΚΡΟΤ, ΠΙΟ, ΔΥΝΑΤ, ΠΡΟΣΛΑΜΒΑΝ]","[(ΤΟ, ΤΕΡΜΑΤΙΣΑΝ), (ΤΕΡΜΑΤΙΣΑΝ, ΣΤΗΝ), (ΣΤΗΝ, ΕΠΙΚΟΙΝΩΝΙΑ), (ΕΠΙΚΟΙΝΩΝΙΑ, Η), (Η, ΧΑΡΑ), (ΧΑΡΑ, ΤΟΥ), (ΤΟΥ, PR), (PR, Η), (Η, ΕΧΟΥΝ), (ΕΧΟΥΝ

## KBL lexicon - nGrams

<li>The number of positive (negative) matches of every unigram and bigram in the lexicons from KBL – 4 features</li>
<li>The total sum (float) of positive (negative) unigrams and bigrams scores from KBL – 4 features</li>
<li>The overall summation of their respective scores from KBL – 2 feature</li>

In [16]:
# create a copy and work on that
df_tweets3 = df_tweets2.copy()

In [17]:
# read the preprocessed KBL lexicon with the Ngrams
df_KBL = pd.read_pickle('/home/infilab/SentimentAnalysis/lexicons/fixed_lexicons/df_KBL.pkl')

In [18]:
df_KBL.head()

,ngram,score,ngram2,ngram_CAP,length
0,ντροπή,-14.280374,ντροπη,ΝΤΡΟΠΗ,1
1,ψώνια,-13.504118,ψωνια,ΨΩΝΙΑ,1
2,μωρή,-13.423770,μωρη,ΜΩΡΗ,1
3,φοβάμαι,-13.406769,φοβαμαι,ΦΟΒΑΜΑΙ,1
4,πόνο,-13.341170,πονο,ΠΟΝΟ,1


In [19]:
# split the df_KBL in two separate dataframe
# one for unigrams
df_KBL_uni = df_KBL[df_KBL.length==1]
# one for bigrams
df_KBL_bi = df_KBL[df_KBL.length==2]

#### -- Unigrams

In [20]:
# create a df with the negative unigrams
df_KBL_uni_negative = df_KBL_uni[df_KBL_uni.score<0].drop_duplicates(subset='ngram_CAP', keep='first')
# create a df with the positive unigrams
df_KBL_uni_positive = df_KBL_uni[df_KBL_uni.score>0].drop_duplicates(subset='ngram_CAP', keep='first')

In [21]:
def n_score_unigrams(terms, df):
    terms_df = pd.DataFrame({'Terms':terms})
    terms_df = terms_df.merge(df[['ngram_CAP','score']], left_on='Terms', right_on='ngram_CAP', how='left')
    terms_df.dropna(inplace=True)
    n = len(terms_df)
    sum_score = terms_df.score.sum()
    
    return [n, sum_score]

In [22]:
# Number of negative unigrams in each tweet and the sum of their scores as list--> [n,score]
df_tweets3['N_Score_Uni_Neg'] = df_tweets3['Tokens'].apply(lambda x: 
                                        n_score_unigrams(x, df_KBL_uni_negative))
# split the list into 2 columns
df_tweets3[['N_Uni_Neg','Score_Uni_Neg']] = pd.DataFrame(df_tweets3.N_Score_Uni_Neg.values.tolist(), index= df_tweets3.index)

In [23]:
# Number of positive unigrams in each tweet and the sum of their scores as list--> [n,score]
df_tweets3['N_Score_Uni_Pos'] = df_tweets3['Tokens'].apply(lambda x: 
                                        n_score_unigrams(x, df_KBL_uni_positive))
# split the list into 2 columns
df_tweets3[['N_Uni_Pos','Score_Uni_Pos']] = pd.DataFrame(df_tweets3.N_Score_Uni_Pos.values.tolist(), index= df_tweets3.index)

In [24]:
# delete these columns
del df_tweets3['N_Score_Uni_Neg']
del df_tweets3['N_Score_Uni_Pos']

#### -- Bigrams

In [25]:
# create a df with the negative bigrams
df_KBL_bi_negative = df_KBL_bi[df_KBL_bi.score<0].drop_duplicates(subset='ngram_CAP', keep='first')
# create a df with the positive bigrams
df_KBL_bi_positive = df_KBL_bi[df_KBL_bi.score>0].drop_duplicates(subset='ngram_CAP', keep='first')

In [26]:
def n_score_bigrams(bigrams, df):
    terms_df = pd.DataFrame({'Bigrams':bigrams})
    terms_df = terms_df.merge(df[['ngram_CAP','score']], left_on='Bigrams', right_on='ngram_CAP', how='left')
    terms_df.dropna(inplace=True)
    n = len(terms_df)
    sum_score = terms_df.score.sum()
    
    return [n, sum_score]

In [27]:
# Number of negative bigrams in each tweet and the sum of their scores as list--> [n,score]
df_tweets3['N_Score_Bi_Neg'] = df_tweets3['Bigrams2'].apply(lambda x: 
                                        n_score_bigrams(x, df_KBL_bi_negative))
# split the list into 2 columns
df_tweets3[['N_Bi_Neg','Score_Bi_Neg']] = pd.DataFrame(df_tweets3.N_Score_Bi_Neg.values.tolist(), index= df_tweets3.index)

In [28]:
# Number of positive bigrams in each tweet and the sum of their scores as list--> [n,score]
df_tweets3['N_Score_Bi_Pos'] = df_tweets3['Bigrams2'].apply(lambda x: 
                                        n_score_bigrams(x, df_KBL_bi_positive))
# split the list into 2 columns
df_tweets3[['N_Bi_Pos','Score_Bi_Pos']] = pd.DataFrame(df_tweets3.N_Score_Bi_Pos.values.tolist(), index= df_tweets3.index)

In [29]:
# delete these columns
del df_tweets3['N_Score_Bi_Neg']
del df_tweets3['N_Score_Bi_Pos']

In [30]:
# The overall summation of their respective scores from KBL – 2 feature
df_tweets3['Score_Uni'] = df_tweets3['Score_Uni_Neg'] + df_tweets3['Score_Uni_Pos']
df_tweets3['Score_Bi'] = df_tweets3['Score_Bi_Neg'] + df_tweets3['Score_Bi_Pos']

In [31]:
df_tweets3.head(2)

,date,hashtags,id,location,mentions,searchParam,text,user_id,user_name,date2,day,Text_only,Text_only2,Text_only3,Tweet_upper,Tokens,Tokens_stem,Bigrams,Bigrams2,N_exclam,N_hash,N_mentions,N_uppers,has_url,N_words,text_id,N_matches,Match_Terms,Sum_subj,Sum_positive,Sum_negative,Sum_anger,Sum_disgust,Sum_fear,Sum_happy,Sum_sad,Sum_surprise,N_Uni_Neg,Score_Uni_Neg,N_Uni_Pos,Score_Uni_Pos,N_Bi_Neg,Score_Bi_Neg,N_Bi_Pos,Score_Bi_Pos,Score_Uni,Score_Bi
0,2019-09-08 23:39:53,[],1170844235215642624,,[EliasPalai],#ΝΔ,RT @EliasPalai: Πρόθεση Ψήφου Εθνικών Εκλογών\n\n(Και κάτοικοι του Εξωτερικού αφού πλέον στις δημοσκοπήσεις θα πρέπει οι εταιρίες δημοσκοπήσε…,1162444088500772864,Μπιλαρος,2019-09-08,4,RT : Πρόθεση Ψήφου Εθνικών Εκλογών (Και κάτοικοι του Εξωτερικού αφού πλέον στις δημοσκοπήσεις θα πρέπει οι εταιρίες δημοσκοπήσε…,rt : πρόθεση ψήφου εθνικών εκλογών (και κάτοικοι του εξωτερικού αφού πλέον στις δημοσκοπήσεις θα πρέπει οι εταιρίες δημοσκοπήσε…,rt : προθεση ψηφου εθνικων εκλογων (και κατοικοι του εξωτερικου αφου πλεον στις δημοσκοπησεις θα πρεπει οι εταιριες δημοσκοπησε…,RT : ΠΡΟΘΕΣΗ ΨΗΦΟΥ ΕΘΝΙΚΩΝ ΕΚΛΟΓΩΝ (ΚΑΙ ΚΑΤΟΙΚΟΙ ΤΟΥ ΕΞΩΤΕΡΙΚΟΥ ΑΦΟΥ ΠΛΕΟΝ ΣΤΙΣ ΔΗΜΟΣΚΟΠΗΣΕΙΣ ΘΑ ΠΡΕΠΕΙ ΟΙ ΕΤΑΙΡΙΕΣ ΔΗΜΟΣΚΟΠΗΣΕ…,"[RT, ΠΡΟΘΕΣΗ, ΨΗΦΟΥ, ΕΘΝΙΚΩΝ, ΕΚΛΟΓΩΝ, ΚΑΙ, ΚΑΤΟΙΚΟΙ, ΤΟΥ, ΕΞΩΤΕΡΙΚΟΥ, ΑΦΟΥ, ΠΛΕΟΝ, ΣΤΙΣ, ΔΗΜΟΣΚΟΠΗΣΕΙΣ, ΘΑ, ΠΡΕΠΕΙ, ΟΙ, ΕΤΑΙΡΙΕΣ, ΔΗΜΟΣΚΟΠΗΣΕ]","[RT, ΠΡΟΘΕΣ, ΨΗΦ, ΕΘΝ, ΕΚΛΟΓ, ΚΑΙ, ΚΑΤΟΙΚ, ΤΟΥ, ΕΞΩΤΕΡ, ΑΦΟΥ, ΠΛΕΟΝ, ΣΤΙΣ, ΔΗΜΟΣΚΟΠΗΣ, ΘΑ, ΠΡΕΠ, ΟΙ, ΕΤΑΙΡ, ΔΗΜΟΣΚΟΠ]","[(RT, ΠΡΟΘΕΣΗ), (ΠΡΟΘΕΣΗ, ΨΗΦΟΥ), (ΨΗΦΟΥ, ΕΘΝΙΚΩΝ), (ΕΘΝΙΚΩΝ, ΕΚΛΟΓΩΝ), (ΕΚΛΟΓΩΝ, ΚΑΙ), (ΚΑΙ, ΚΑΤΟΙΚΟΙ), (ΚΑΤΟΙΚΟΙ, ΤΟΥ), (ΤΟΥ, ΕΞΩΤΕΡΙΚΟΥ), (ΕΞΩΤΕΡΙΚΟΥ, ΑΦΟΥ), (ΑΦΟΥ, ΠΛΕΟΝ), (ΠΛΕΟΝ, ΣΤΙΣ), (ΣΤΙΣ, ΔΗΜΟΣΚΟΠΗΣΕΙΣ), (ΔΗΜΟΣΚΟΠΗΣΕΙΣ, ΘΑ), (ΘΑ, ΠΡΕΠΕΙ), (ΠΡΕΠΕΙ, ΟΙ), (ΟΙ, ΕΤΑΙΡΙΕΣ), (ΕΤΑΙΡΙΕΣ, ΔΗΜΟΣΚΟΠΗΣΕ)]","[RT ΠΡΟΘΕΣΗ, ΠΡΟΘΕΣΗ ΨΗΦΟΥ, ΨΗΦΟΥ ΕΘΝΙΚΩΝ, ΕΘΝΙΚΩΝ ΕΚΛΟΓΩΝ, ΕΚΛΟΓΩΝ ΚΑΙ, ΚΑΙ ΚΑΤΟΙΚΟΙ, ΚΑΤΟΙΚΟΙ ΤΟΥ, ΤΟΥ ΕΞΩΤΕΡΙΚΟΥ, ΕΞΩΤΕΡΙΚΟΥ ΑΦΟΥ, ΑΦΟΥ ΠΛΕΟΝ, ΠΛΕΟΝ ΣΤΙΣ, ΣΤΙΣ ΔΗΜΟΣΚΟΠΗΣΕΙΣ, ΔΗΜΟΣΚΟΠΗΣΕΙΣ ΘΑ, ΘΑ ΠΡΕΠΕΙ, ΠΡΕΠΕΙ ΟΙ, ΟΙ ΕΤΑΙΡΙΕΣ, ΕΤΑΙΡΙΕΣ ΔΗΜΟΣΚΟΠΗΣΕ]",0,0,1,1,0,18,690,2,[ΤΟΥ],0.125,0.0,0.25,0.0,0.0000,0.0000,0.0000,0.0,0.0,15,-11.100170,4,2.192205,6,-4.741375,1,0.483590,-8.907965,-4.257785
1,2019-09-08 22:46:36,"[ΝΔ, Επιτελικό_κράτος, Μητσοτακη, ΔΕΘ2019, Βελλιδειο]",1170830825153318912,"Athens, Greece",[],#ΝΔ,"Το τερμάτισαν στην ""επικοινωνία"" (η χάρα του PR η #ΝΔ, έχουν και γνωστούς με γραφεία...). Καμιά κατοστάρια μόνο δημοσιογράφους θα προσλάβει το #Επιτελικό_κράτος (κομαντατούρα #Μητσοτακη). Πήγαν για interview; Οποιος χειροκροτήσει πιο δυνατά προσλαμβάνεται;😂 #ΔΕΘ2019 #Βελλιδειο https://t.co/2duOrmL7u5",1142893644082503681,Παναγιώτης Χατζηπλής 🇬🇷,2019-09-08,4,"Το τερμάτισαν στην ""επικοινωνία"" (η χάρα του PR η , έχουν και γνωστούς με γραφεία...). Καμιά κατοστάρια μόνο δημοσιογράφους θα προσλάβει το (κομαντατούρα ). Πήγαν για interview; Οποιος χειροκροτήσει πιο δυνατά προσλαμβάνεται;","το τερμάτισαν στην ""επικοινωνία"" (η χάρα του pr η , έχουν και γνωστούς με γραφεία...). καμιά κατοστάρια μόνο δημοσιογράφους θα προσλάβει το (κομαντατούρα ). πήγαν για interview; οποιος χειροκροτήσει πιο δυνατά προσλαμβάνεται;","το τερματισαν στην ""επικοινωνια"" (η χαρα του pr η , εχουν και γνωστους με γραφεια...). καμια κατοσταρια μονο δημοσιογραφους θα προσλαβει το (κομαντατουρα ). πηγαν για interview; οποιος χειροκροτησει πιο δυνατα προσλαμβανεται;","ΤΟ ΤΕΡΜΑΤΙΣΑΝ ΣΤΗΝ ""ΕΠΙΚΟΙΝΩΝΙΑ"" (Η ΧΑΡΑ ΤΟΥ PR Η , ΕΧΟΥΝ ΚΑΙ ΓΝΩΣΤΟΥΣ ΜΕ ΓΡΑΦΕΙΑ...). ΚΑΜΙΑ ΚΑΤΟΣΤΑΡΙΑ ΜΟΝΟ ΔΗΜΟΣΙΟΓΡΑΦΟΥΣ ΘΑ ΠΡΟΣΛΑΒΕΙ ΤΟ (ΚΟΜΑΝΤΑΤΟΥΡΑ ). ΠΗΓΑΝ ΓΙΑ INTERVIEW; ΟΠΟΙΟΣ ΧΕΙΡΟΚΡΟΤΗΣΕΙ ΠΙΟ ΔΥΝΑΤΑ ΠΡΟΣΛΑΜΒΑΝΕΤΑΙ;","[ΤΟ, ΤΕΡΜΑΤΙΣΑΝ, ΣΤΗΝ, ΕΠΙΚΟΙΝΩΝΙΑ, Η, ΧΑΡΑ, ΤΟΥ, PR, Η, ΕΧΟΥΝ, ΚΑΙ, ΓΝΩΣΤΟΥΣ, ΜΕ, ΓΡΑΦΕΙΑ, ΚΑΜΙΑ, ΚΑΤΟΣΤΑΡΙΑ, ΜΟΝΟ, ΔΗΜΟΣΙΟΓΡΑΦΟΥΣ, ΘΑ, ΠΡΟΣΛΑΒΕΙ, ΤΟ, ΚΟΜΑΝΤΑΤΟΥΡΑ, ΠΗΓΑΝ, ΓΙΑ, INTERVIEW, ΟΠΟΙΟΣ, ΧΕΙΡΟΚΡΟΤΗΣΕΙ, ΠΙΟ, ΔΥΝΑΤΑ, ΠΡΟΣΛΑΜΒΑΝΕΤΑΙ]","[ΤΟ, ΤΕΡΜΑΤΙΣ, ΣΤΗΝ, ΕΠΙΚΟΙΝΩΝ, Η, ΧΑΡ, ΤΟΥ, PR, Η, ΕΧ, ΚΑΙ, ΓΝΩΣΤ, ΜΕ, ΓΡΑΦΕΙ, ΚΑΜ, ΚΑΤΟΣΤΑΡ, ΜΟΝ, ΔΗΜΟΣΙΟΓΡΑΦ, ΘΑ, ΠΡΟΣΛΑΒ, ΤΟ, ΚΟΜΑΝΤΑΤΟΥΡ, ΠΗΓ, ΓΙΑ, INTERVIEW, ΟΠΟΙ

In [32]:
df_tweets3.shape

(3172, 47)

In [33]:
df_tweets3.columns

Index(['date', 'hashtags', 'id', 'location', 'mentions', 'searchParam', 'text',
       'user_id', 'user_name', 'date2', 'day', 'Text_only', 'Text_only2',
       'Text_only3', 'Tweet_upper', 'Tokens', 'Tokens_stem', 'Bigrams',
       'Bigrams2', 'N_exclam', 'N_hash', 'N_mentions', 'N_uppers', 'has_url',
       'N_words', 'text_id', 'N_matches', 'Match_Terms', 'Sum_subj',
       'Sum_positive', 'Sum_negative', 'Sum_anger', 'Sum_disgust', 'Sum_fear',
       'Sum_happy', 'Sum_sad', 'Sum_surprise', 'N_Uni_Neg', 'Score_Uni_Neg',
       'N_Uni_Pos', 'Score_Uni_Pos', 'N_Bi_Neg', 'Score_Bi_Neg', 'N_Bi_Pos',
       'Score_Bi_Pos', 'Score_Uni', 'Score_Bi'],
      dtype='object')

In [34]:
# keep specific columns
df_tweets4 = df_tweets3[['text_id', 'N_matches',
       'Match_Terms', 'Sum_subj', 'Sum_positive', 'Sum_negative', 'Sum_anger',
       'Sum_disgust', 'Sum_fear', 'Sum_happy', 'Sum_sad', 'Sum_surprise',
       'N_Uni_Neg', 'Score_Uni_Neg', 'N_Uni_Pos', 'Score_Uni_Pos', 'N_Bi_Neg',
       'Score_Bi_Neg', 'N_Bi_Pos', 'Score_Bi_Pos', 'Score_Uni', 'Score_Bi']]

In [35]:
df_tweets4.text_id.nunique()

3172

In [36]:
# merge the above df with the initital df_tweets
df_tweets4 = df_tweets.merge(df_tweets4, left_on='text_id', right_on='text_id', how='left')

In [37]:
df_tweets4.shape

(11324, 47)

In [38]:
df_tweets4.head(2)

,date,hashtags,id,location,mentions,searchParam,text,user_id,user_name,date2,day,Text_only,Text_only2,Text_only3,Tweet_upper,Tokens,Tokens_stem,Bigrams,Bigrams2,N_exclam,N_hash,N_mentions,N_uppers,has_url,N_words,text_id,N_matches,Match_Terms,Sum_subj,Sum_positive,Sum_negative,Sum_anger,Sum_disgust,Sum_fear,Sum_happy,Sum_sad,Sum_surprise,N_Uni_Neg,Score_Uni_Neg,N_Uni_Pos,Score_Uni_Pos,N_Bi_Neg,Score_Bi_Neg,N_Bi_Pos,Score_Bi_Pos,Score_Uni,Score_Bi
0,2019-09-08 23:39:53,[],1170844235215642624,,[EliasPalai],#ΝΔ,RT @EliasPalai: Πρόθεση Ψήφου Εθνικών Εκλογών\n\n(Και κάτοικοι του Εξωτερικού αφού πλέον στις δημοσκοπήσεις θα πρέπει οι εταιρίες δημοσκοπήσε…,1162444088500772864,Μπιλαρος,2019-09-08,4,RT : Πρόθεση Ψήφου Εθνικών Εκλογών (Και κάτοικοι του Εξωτερικού αφού πλέον στις δημοσκοπήσεις θα πρέπει οι εταιρίες δημοσκοπήσε…,rt : πρόθεση ψήφου εθνικών εκλογών (και κάτοικοι του εξωτερικού αφού πλέον στις δημοσκοπήσεις θα πρέπει οι εταιρίες δημοσκοπήσε…,rt : προθεση ψηφου εθνικων εκλογων (και κατοικοι του εξωτερικου αφου πλεον στις δημοσκοπησεις θα πρεπει οι εταιριες δημοσκοπησε…,RT : ΠΡΟΘΕΣΗ ΨΗΦΟΥ ΕΘΝΙΚΩΝ ΕΚΛΟΓΩΝ (ΚΑΙ ΚΑΤΟΙΚΟΙ ΤΟΥ ΕΞΩΤΕΡΙΚΟΥ ΑΦΟΥ ΠΛΕΟΝ ΣΤΙΣ ΔΗΜΟΣΚΟΠΗΣΕΙΣ ΘΑ ΠΡΕΠΕΙ ΟΙ ΕΤΑΙΡΙΕΣ ΔΗΜΟΣΚΟΠΗΣΕ…,"[RT, ΠΡΟΘΕΣΗ, ΨΗΦΟΥ, ΕΘΝΙΚΩΝ, ΕΚΛΟΓΩΝ, ΚΑΙ, ΚΑΤΟΙΚΟΙ, ΤΟΥ, ΕΞΩΤΕΡΙΚΟΥ, ΑΦΟΥ, ΠΛΕΟΝ, ΣΤΙΣ, ΔΗΜΟΣΚΟΠΗΣΕΙΣ, ΘΑ, ΠΡΕΠΕΙ, ΟΙ, ΕΤΑΙΡΙΕΣ, ΔΗΜΟΣΚΟΠΗΣΕ]","[RT, ΠΡΟΘΕΣ, ΨΗΦ, ΕΘΝ, ΕΚΛΟΓ, ΚΑΙ, ΚΑΤΟΙΚ, ΤΟΥ, ΕΞΩΤΕΡ, ΑΦΟΥ, ΠΛΕΟΝ, ΣΤΙΣ, ΔΗΜΟΣΚΟΠΗΣ, ΘΑ, ΠΡΕΠ, ΟΙ, ΕΤΑΙΡ, ΔΗΜΟΣΚΟΠ]","[(RT, ΠΡΟΘΕΣΗ), (ΠΡΟΘΕΣΗ, ΨΗΦΟΥ), (ΨΗΦΟΥ, ΕΘΝΙΚΩΝ), (ΕΘΝΙΚΩΝ, ΕΚΛΟΓΩΝ), (ΕΚΛΟΓΩΝ, ΚΑΙ), (ΚΑΙ, ΚΑΤΟΙΚΟΙ), (ΚΑΤΟΙΚΟΙ, ΤΟΥ), (ΤΟΥ, ΕΞΩΤΕΡΙΚΟΥ), (ΕΞΩΤΕΡΙΚΟΥ, ΑΦΟΥ), (ΑΦΟΥ, ΠΛΕΟΝ), (ΠΛΕΟΝ, ΣΤΙΣ), (ΣΤΙΣ, ΔΗΜΟΣΚΟΠΗΣΕΙΣ), (ΔΗΜΟΣΚΟΠΗΣΕΙΣ, ΘΑ), (ΘΑ, ΠΡΕΠΕΙ), (ΠΡΕΠΕΙ, ΟΙ), (ΟΙ, ΕΤΑΙΡΙΕΣ), (ΕΤΑΙΡΙΕΣ, ΔΗΜΟΣΚΟΠΗΣΕ)]","[RT ΠΡΟΘΕΣΗ, ΠΡΟΘΕΣΗ ΨΗΦΟΥ, ΨΗΦΟΥ ΕΘΝΙΚΩΝ, ΕΘΝΙΚΩΝ ΕΚΛΟΓΩΝ, ΕΚΛΟΓΩΝ ΚΑΙ, ΚΑΙ ΚΑΤΟΙΚΟΙ, ΚΑΤΟΙΚΟΙ ΤΟΥ, ΤΟΥ ΕΞΩΤΕΡΙΚΟΥ, ΕΞΩΤΕΡΙΚΟΥ ΑΦΟΥ, ΑΦΟΥ ΠΛΕΟΝ, ΠΛΕΟΝ ΣΤΙΣ, ΣΤΙΣ ΔΗΜΟΣΚΟΠΗΣΕΙΣ, ΔΗΜΟΣΚΟΠΗΣΕΙΣ ΘΑ, ΘΑ ΠΡΕΠΕΙ, ΠΡΕΠΕΙ ΟΙ, ΟΙ ΕΤΑΙΡΙΕΣ, ΕΤΑΙΡΙΕΣ ΔΗΜΟΣΚΟΠΗΣΕ]",0,0,1,1,0,18,690,2,[ΤΟΥ],0.125,0.0,0.25,0.0,0.0000,0.0000,0.0000,0.0,0.0,15,-11.100170,4,2.192205,6,-4.741375,1,0.483590,-8.907965,-4.257785
1,2019-09-08 22:46:36,"[ΝΔ, Επιτελικό_κράτος, Μητσοτακη, ΔΕΘ2019, Βελλιδειο]",1170830825153318912,"Athens, Greece",[],#ΝΔ,"Το τερμάτισαν στην ""επικοινωνία"" (η χάρα του PR η #ΝΔ, έχουν και γνωστούς με γραφεία...). Καμιά κατοστάρια μόνο δημοσιογράφους θα προσλάβει το #Επιτελικό_κράτος (κομαντατούρα #Μητσοτακη). Πήγαν για interview; Οποιος χειροκροτήσει πιο δυνατά προσλαμβάνεται;😂 #ΔΕΘ2019 #Βελλιδειο https://t.co/2duOrmL7u5",1142893644082503681,Παναγιώτης Χατζηπλής 🇬🇷,2019-09-08,4,"Το τερμάτισαν στην ""επικοινωνία"" (η χάρα του PR η , έχουν και γνωστούς με γραφεία...). Καμιά κατοστάρια μόνο δημοσιογράφους θα προσλάβει το (κομαντατούρα ). Πήγαν για interview; Οποιος χειροκροτήσει πιο δυνατά προσλαμβάνεται;","το τερμάτισαν στην ""επικοινωνία"" (η χάρα του pr η , έχουν και γνωστούς με γραφεία...). καμιά κατοστάρια μόνο δημοσιογράφους θα προσλάβει το (κομαντατούρα ). πήγαν για interview; οποιος χειροκροτήσει πιο δυνατά προσλαμβάνεται;","το τερματισαν στην ""επικοινωνια"" (η χαρα του pr η , εχουν και γνωστους με γραφεια...). καμια κατοσταρια μονο δημοσιογραφους θα προσλαβει το (κομαντατουρα ). πηγαν για interview; οποιος χειροκροτησει πιο δυνατα προσλαμβανεται;","ΤΟ ΤΕΡΜΑΤΙΣΑΝ ΣΤΗΝ ""ΕΠΙΚΟΙΝΩΝΙΑ"" (Η ΧΑΡΑ ΤΟΥ PR Η , ΕΧΟΥΝ ΚΑΙ ΓΝΩΣΤΟΥΣ ΜΕ ΓΡΑΦΕΙΑ...). ΚΑΜΙΑ ΚΑΤΟΣΤΑΡΙΑ ΜΟΝΟ ΔΗΜΟΣΙΟΓΡΑΦΟΥΣ ΘΑ ΠΡΟΣΛΑΒΕΙ ΤΟ (ΚΟΜΑΝΤΑΤΟΥΡΑ ). ΠΗΓΑΝ ΓΙΑ INTERVIEW; ΟΠΟΙΟΣ ΧΕΙΡΟΚΡΟΤΗΣΕΙ ΠΙΟ ΔΥΝΑΤΑ ΠΡΟΣΛΑΜΒΑΝΕΤΑΙ;","[ΤΟ, ΤΕΡΜΑΤΙΣΑΝ, ΣΤΗΝ, ΕΠΙΚΟΙΝΩΝΙΑ, Η, ΧΑΡΑ, ΤΟΥ, PR, Η, ΕΧΟΥΝ, ΚΑΙ, ΓΝΩΣΤΟΥΣ, ΜΕ, ΓΡΑΦΕΙΑ, ΚΑΜΙΑ, ΚΑΤΟΣΤΑΡΙΑ, ΜΟΝΟ, ΔΗΜΟΣΙΟΓΡΑΦΟΥΣ, ΘΑ, ΠΡΟΣΛΑΒΕΙ, ΤΟ, ΚΟΜΑΝΤΑΤΟΥΡΑ, ΠΗΓΑΝ, ΓΙΑ, INTERVIEW, ΟΠΟΙΟΣ, ΧΕΙΡΟΚΡΟΤΗΣΕΙ, ΠΙΟ, ΔΥΝΑΤΑ, ΠΡΟΣΛΑΜΒΑΝΕΤΑΙ]","[ΤΟ, ΤΕΡΜΑΤΙΣ, ΣΤΗΝ, ΕΠΙΚΟΙΝΩΝ, Η, ΧΑΡ, ΤΟΥ, PR, Η, ΕΧ, ΚΑΙ, ΓΝΩΣΤ, ΜΕ, ΓΡΑΦΕΙ, ΚΑΜ, ΚΑΤΟΣΤΑΡ, ΜΟΝ, ΔΗΜΟΣΙΟΓΡΑΦ, ΘΑ, ΠΡΟΣΛΑΒ, ΤΟ, ΚΟΜΑΝΤΑΤΟΥΡ, ΠΗΓ, ΓΙΑ, INTERVIEW, ΟΠΟΙ

In [39]:
# save the df as pickle file
df_tweets4.to_pickle('results/df_tweets_features_lexicons_v1.pkl')

### Features based on spacy

In [40]:
# read the file that was created in tweets_syntax.ipynb
df_tweets_pos = pd.read_pickle('results/tweets_step_2_v1.pkl')

In [41]:
# drop duplicates
df_tweets_pos = df_tweets_pos.drop_duplicates(subset=['id'], keep='first')

In [42]:
df_tweets_pos.head()

,id,N_verbs,N_adjs,N_nouns
0,1170844235215642624,1,3,8
1,1170830825153318912,6,2,7
2,1170830212986232832,1,1,6
3,1170830138436665344,1,1,6
4,1170824769924665344,3,1,5


In [43]:
df_tweets_pos.shape

(9571, 4)

### Create final dataset by merging them

In [44]:
# add the information from spacy to the main dataset
df_final = df_tweets4.merge(df_tweets_pos, left_on='id', right_on='id', how='left')

In [45]:
df_final.shape

(11324, 50)

In [46]:
df_final.head(7)

,date,hashtags,id,location,mentions,searchParam,text,user_id,user_name,date2,day,Text_only,Text_only2,Text_only3,Tweet_upper,Tokens,Tokens_stem,Bigrams,Bigrams2,N_exclam,N_hash,N_mentions,N_uppers,has_url,N_words,text_id,N_matches,Match_Terms,Sum_subj,Sum_positive,Sum_negative,Sum_anger,Sum_disgust,Sum_fear,Sum_happy,Sum_sad,Sum_surprise,N_Uni_Neg,Score_Uni_Neg,N_Uni_Pos,Score_Uni_Pos,N_Bi_Neg,Score_Bi_Neg,N_Bi_Pos,Score_Bi_Pos,Score_Uni,Score_Bi,N_verbs,N_adjs,N_nouns
0,2019-09-08 23:39:53,[],1170844235215642624,,[EliasPalai],#ΝΔ,RT @EliasPalai: Πρόθεση Ψήφου Εθνικών Εκλογών\n\n(Και κάτοικοι του Εξωτερικού αφού πλέον στις δημοσκοπήσεις θα πρέπει οι εταιρίες δημοσκοπήσε…,1162444088500772864,Μπιλαρος,2019-09-08,4,RT : Πρόθεση Ψήφου Εθνικών Εκλογών (Και κάτοικοι του Εξωτερικού αφού πλέον στις δημοσκοπήσεις θα πρέπει οι εταιρίες δημοσκοπήσε…,rt : πρόθεση ψήφου εθνικών εκλογών (και κάτοικοι του εξωτερικού αφού πλέον στις δημοσκοπήσεις θα πρέπει οι εταιρίες δημοσκοπήσε…,rt : προθεση ψηφου εθνικων εκλογων (και κατοικοι του εξωτερικου αφου πλεον στις δημοσκοπησεις θα πρεπει οι εταιριες δημοσκοπησε…,RT : ΠΡΟΘΕΣΗ ΨΗΦΟΥ ΕΘΝΙΚΩΝ ΕΚΛΟΓΩΝ (ΚΑΙ ΚΑΤΟΙΚΟΙ ΤΟΥ ΕΞΩΤΕΡΙΚΟΥ ΑΦΟΥ ΠΛΕΟΝ ΣΤΙΣ ΔΗΜΟΣΚΟΠΗΣΕΙΣ ΘΑ ΠΡΕΠΕΙ ΟΙ ΕΤΑΙΡΙΕΣ ΔΗΜΟΣΚΟΠΗΣΕ…,"[RT, ΠΡΟΘΕΣΗ, ΨΗΦΟΥ, ΕΘΝΙΚΩΝ, ΕΚΛΟΓΩΝ, ΚΑΙ, ΚΑΤΟΙΚΟΙ, ΤΟΥ, ΕΞΩΤΕΡΙΚΟΥ, ΑΦΟΥ, ΠΛΕΟΝ, ΣΤΙΣ, ΔΗΜΟΣΚΟΠΗΣΕΙΣ, ΘΑ, ΠΡΕΠΕΙ, ΟΙ, ΕΤΑΙΡΙΕΣ, ΔΗΜΟΣΚΟΠΗΣΕ]","[RT, ΠΡΟΘΕΣ, ΨΗΦ, ΕΘΝ, ΕΚΛΟΓ, ΚΑΙ, ΚΑΤΟΙΚ, ΤΟΥ, ΕΞΩΤΕΡ, ΑΦΟΥ, ΠΛΕΟΝ, ΣΤΙΣ, ΔΗΜΟΣΚΟΠΗΣ, ΘΑ, ΠΡΕΠ, ΟΙ, ΕΤΑΙΡ, ΔΗΜΟΣΚΟΠ]","[(RT, ΠΡΟΘΕΣΗ), (ΠΡΟΘΕΣΗ, ΨΗΦΟΥ), (ΨΗΦΟΥ, ΕΘΝΙΚΩΝ), (ΕΘΝΙΚΩΝ, ΕΚΛΟΓΩΝ), (ΕΚΛΟΓΩΝ, ΚΑΙ), (ΚΑΙ, ΚΑΤΟΙΚΟΙ), (ΚΑΤΟΙΚΟΙ, ΤΟΥ), (ΤΟΥ, ΕΞΩΤΕΡΙΚΟΥ), (ΕΞΩΤΕΡΙΚΟΥ, ΑΦΟΥ), (ΑΦΟΥ, ΠΛΕΟΝ), (ΠΛΕΟΝ, ΣΤΙΣ), (ΣΤΙΣ, ΔΗΜΟΣΚΟΠΗΣΕΙΣ), (ΔΗΜΟΣΚΟΠΗΣΕΙΣ, ΘΑ), (ΘΑ, ΠΡΕΠΕΙ), (ΠΡΕΠΕΙ, ΟΙ), (ΟΙ, ΕΤΑΙΡΙΕΣ), (ΕΤΑΙΡΙΕΣ, ΔΗΜΟΣΚΟΠΗΣΕ)]","[RT ΠΡΟΘΕΣΗ, ΠΡΟΘΕΣΗ ΨΗΦΟΥ, ΨΗΦΟΥ ΕΘΝΙΚΩΝ, ΕΘΝΙΚΩΝ ΕΚΛΟΓΩΝ, ΕΚΛΟΓΩΝ ΚΑΙ, ΚΑΙ ΚΑΤΟΙΚΟΙ, ΚΑΤΟΙΚΟΙ ΤΟΥ, ΤΟΥ ΕΞΩΤΕΡΙΚΟΥ, ΕΞΩΤΕΡΙΚΟΥ ΑΦΟΥ, ΑΦΟΥ ΠΛΕΟΝ, ΠΛΕΟΝ ΣΤΙΣ, ΣΤΙΣ ΔΗΜΟΣΚΟΠΗΣΕΙΣ, ΔΗΜΟΣΚΟΠΗΣΕΙΣ ΘΑ, ΘΑ ΠΡΕΠΕΙ, ΠΡΕΠΕΙ ΟΙ, ΟΙ ΕΤΑΙΡΙΕΣ, ΕΤΑΙΡΙΕΣ ΔΗΜΟΣΚΟΠΗΣΕ]",0,0,1,1,0,18,690,2,[ΤΟΥ],0.125,0.00,0.25,0.0,0.0000,0.0000,0.0000,0.0,0.00,15,-11.100170,4,2.192205,6,-4.741375,1,0.483590,-8.907965,-4.257785,1,3,8
1,2019-09-08 22:46:36,"[ΝΔ, Επιτελικό_κράτος, Μητσοτακη, ΔΕΘ2019, Βελλιδειο]",1170830825153318912,"Athens, Greece",[],#ΝΔ,"Το τερμάτισαν στην ""επικοινωνία"" (η χάρα του PR η #ΝΔ, έχουν και γνωστούς με γραφεία...). Καμιά κατοστάρια μόνο δημοσιογράφους θα προσλάβει το #Επιτελικό_κράτος (κομαντατούρα #Μητσοτακη). Πήγαν για interview; Οποιος χειροκροτήσει πιο δυνατά προσλαμβάνεται;😂 #ΔΕΘ2019 #Βελλιδειο https://t.co/2duOrmL7u5",1142893644082503681,Παναγιώτης Χατζηπλής 🇬🇷,2019-09-08,4,"Το τερμάτισαν στην ""επικοινωνία"" (η χάρα του PR η , έχουν και γνωστούς με γραφεία...). Καμιά κατοστάρια μόνο δημοσιογράφους θα προσλάβει το (κομαντατούρα ). Πήγαν για interview; Οποιος χειροκροτήσει πιο δυνατά προσλαμβάνεται;","το τερμάτισαν στην ""επικοινωνία"" (η χάρα του pr η , έχουν και γνωστούς με γραφεία...). καμιά κατοστάρια μόνο δημοσιογράφους θα προσλάβει το (κομαντατούρα ). πήγαν για interview; οποιος χειροκροτήσει πιο δυνατά προσλαμβάνεται;","το τερματισαν στην ""επικοινωνια"" (η χαρα του pr η , εχουν και γνωστους με γραφεια...). καμια κατοσταρια μονο δημοσιογραφους θα προσλαβει το (κομαντατουρα ). πηγαν για interview; οποιος χειροκροτησει πιο δυνατα προσλαμβανεται;","ΤΟ ΤΕΡΜΑΤΙΣΑΝ ΣΤΗΝ ""ΕΠΙΚΟΙΝΩΝΙΑ"" (Η ΧΑΡΑ ΤΟΥ PR Η , ΕΧΟΥΝ ΚΑΙ ΓΝΩΣΤΟΥΣ ΜΕ ΓΡΑΦΕΙΑ...). ΚΑΜΙΑ ΚΑΤΟΣΤΑΡΙΑ ΜΟΝΟ ΔΗΜΟΣΙΟΓΡΑΦΟΥΣ ΘΑ ΠΡΟΣΛΑΒΕΙ ΤΟ (ΚΟΜΑΝΤΑΤΟΥΡΑ ). ΠΗΓΑΝ ΓΙΑ INTERVIEW; ΟΠΟΙΟΣ ΧΕΙΡΟΚΡΟΤΗΣΕΙ ΠΙΟ ΔΥΝΑΤΑ ΠΡΟΣΛΑΜΒΑΝΕΤΑΙ;","[ΤΟ, ΤΕΡΜΑΤΙΣΑΝ, ΣΤΗΝ, ΕΠΙΚΟΙΝΩΝΙΑ, Η, ΧΑΡΑ, ΤΟΥ, PR, Η, ΕΧΟΥΝ, ΚΑΙ, ΓΝΩΣΤΟΥΣ, ΜΕ, ΓΡΑΦΕΙΑ, ΚΑΜΙΑ, ΚΑΤΟΣΤΑΡΙΑ, ΜΟΝΟ, ΔΗΜΟΣΙΟΓΡΑΦΟΥΣ, ΘΑ, ΠΡΟΣΛΑΒΕΙ, ΤΟ, ΚΟΜΑΝΤΑΤΟΥΡΑ, ΠΗΓΑΝ, ΓΙΑ, INTERVIEW, ΟΠΟΙΟΣ, ΧΕΙΡΟΚΡΟΤΗΣΕΙ, ΠΙΟ, ΔΥΝΑΤΑ, ΠΡΟΣΛΑΜΒΑΝΕΤΑΙ]","[ΤΟ, ΤΕΡΜΑΤΙΣ, ΣΤΗΝ, ΕΠΙΚΟΙΝΩΝ, Η, ΧΑΡ, ΤΟΥ, PR, Η, ΕΧ, ΚΑΙ, ΓΝΩΣΤ, ΜΕ, ΓΡΑΦΕΙ, ΚΑΜ, ΚΑΤΟΣΤΑΡ, ΜΟΝ, ΔΗΜΟΣΙΟΓΡΑΦ, ΘΑ, ΠΡΟΣΛΑΒ, ΤΟ, ΚΟΜΑΝΤΑ

In [47]:
df_final.shape

(11324, 50)

In [48]:
# save the final dataset as pickle file
df_final.to_pickle('results/df_final2_v1.pkl')